# Use the pocket file and ProtGVP_input to get the final input

In [ ]:
import jsonlines
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder 

In [ ]:
def get_onehot(path,name,encoder):
    res_code = []
    res_coors = []
    with open(os.path.join(path,name)) as f:
        length = 0
        line = f.readline().strip()
        while line:
            lines = line.split(" ")
            res_xyz=[]
            length = length + 1
            res =np.array([lines[0]])
            #print(line[0])
            x=float(lines[1])
            y=float(lines[2])
            z=float(lines[3])
#             except:
#                 print(line)
#                 return  res_code,res_coors
            res_xyz.append(x)
            res_xyz.append(y)
            res_xyz.append(z)

            res_code.append(res)
            res_coors.append(res_xyz)

            line = f.readline().strip()  
        #The coordinates define the type and padding
        res_coors = np.array(res_coors).reshape(len(res_coors), -1)
        #print(len(res_coors))
        #The encoding defines the type and padding
        res_code = encoder.transform(res_code).toarray().astype(np.float32)
    return res_code,res_coors

In [ ]:

now_num =0
all_data = 18100
max_pro_length = 50
gvpdata_file = './gvp_data6A.jsonl'


res_names_origin = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
res_names = np.array(res_names_origin).reshape(len(res_names_origin), -1)
#One Hot
enc = OneHotEncoder()
enc.fit(res_names)

pro_path = '../pocket_acid6A/'
with jsonlines.open('se3_mynet_c6.jsonl', mode='w') as writer: 
    with open(gvpdata_file, "r+", encoding="utf8") as f:
        for gvp_data in jsonlines.Reader(f):
            now_num=now_num+1
            print("\r", end="")
            print("Download progress: {}% ".format(now_num/all_data*100), end="")
            item={}
            item['name'] = gvp_data['name'] 
            
            
            pro_feature,pro_coords =  get_onehot(pro_path,item['name'],enc)
            
            pro_length = len(pro_feature)
            # n*1024  (np.float32)会损失精度
            #pro_codes = np.array(pro_data).reshape(-1, 1024)
            pro_code = np.pad(pro_feature,((0,max_pro_length-pro_length),(0,0)),'constant',constant_values=(0,0)).astype(np.float32)
            pro_coords_np = np.pad(pro_coords,((0,max_pro_length-pro_length),(0,0)),'constant',constant_values=(0,0)).astype(np.float32)

            se3_mask = np.array(gvp_data['pro_mask']).astype(np.bool)

            item['pro_feature'] =   pro_code.tolist()
            item['pro_coords'] = pro_coords_np.tolist()
            
            
            
            item['se3_mask'] = se3_mask.tolist()
            item['pro_mask'] = gvp_data['pro_mask']
            item['pep_feature'] = gvp_data['pep_feature']
            item['pep_mask'] = gvp_data['pep_mask']
            item['label'] = gvp_data['label']
            writer.write(item)